# Black Scholes Exercise 2: NumPy implementation

- Use cProfile and Line Profiler to look for bottlenecks and hotspots in the code

In [1]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np
%load_ext line_profiler

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

# The NumPy modified naive Black Scholes algorithm (looped)

- Minimally converted code from the Naive example
- TODO: Convert the math import to numpy variants

In [13]:
from math import log, sqrt, exp, erf
invsqrt = lambda x: 1.0/sqrt(x)

def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = price[i]
        S = strike[i]
        T = t[i]
        
        a = log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * erf(w1)
        d2 = 0.5 + 0.5 * erf(w2)
        
        Se = exp(b) * S
        
        call[i] = P * d1 - Se * d2
        put[i] = call[i] - P + Se
        #print(call,put)
        
import numpy as np        
np_invsqrt = lambda x: 1.0/np.sqrt(x)
import scipy.special as spec        
    
def np_black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    mr = -rate
    sig_sig_two = vol * vol * 2
    
    for i in range(nopt):
        P = price[i]
        S = strike[i]
        T = t[i]
        
        a = np.log(P / S)
        b = T * mr
        
        z = T * sig_sig_two
        c = 0.25 * z
        y = np_invsqrt(z)
        
        w1 = (a - b + c) * y
        w2 = (a - b - c) * y
        
        d1 = 0.5 + 0.5 * spec.erf(w1)
        d2 = 0.5 + 0.5 * spec.erf(w2)
        
        Se = np.exp(b) * S
        
        call[i] = P * d1 - Se * d2
        put[i] = call[i] - P + Se
        #print(call,put)


## Run timeit, cProfile, and line_profiler to see what is happening

Python Version

In [11]:
%timeit black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

306 ms ± 8.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
cProfile.run('black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)')

         600004 function calls in 0.438 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.026    0.000    0.038    0.000 <ipython-input-13-1782969c5f48>:2(<lambda>)
        1    0.304    0.304    0.438    0.438 <ipython-input-13-1782969c5f48>:4(black_scholes)
        1    0.000    0.000    0.438    0.438 <string>:1(<module>)
        1    0.000    0.000    0.438    0.438 {built-in method builtins.exec}
   200000    0.063    0.000    0.063    0.000 {built-in method math.erf}
   100000    0.013    0.000    0.013    0.000 {built-in method math.exp}
   100000    0.020    0.000    0.020    0.000 {built-in method math.log}
   100000    0.012    0.000    0.012    0.000 {built-in method math.sqrt}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [17]:
%lprun -f black_scholes black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

Timer unit: 1e-06 s

Total time: 1.6443 s
File: <ipython-input-13-1782969c5f48>
Function: black_scholes at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
     5         1            5      5.0      0.0      mr = -rate
     6         1            2      2.0      0.0      sig_sig_two = vol * vol * 2
     7                                               
     8    100001        80425      0.8      4.9      for i in range(nopt):
     9    100000        87204      0.9      5.3          P = price[i]
    10    100000        92621      0.9      5.6          S = strike[i]
    11    100000        87341      0.9      5.3          T = t[i]
    12                                                   
    13    100000       105922      1.1      6.4          a = log(P / S)
    14    100000        85456      0.9      5.2          b = T * mr
    15                    

Numpy Version

In [14]:
%timeit np_black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

720 ms ± 8.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
cProfile.run('np_black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)')

         100004 function calls in 0.742 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.108    0.000    0.108    0.000 <ipython-input-13-1782969c5f48>:33(<lambda>)
        1    0.635    0.635    0.742    0.742 <ipython-input-13-1782969c5f48>:36(np_black_scholes)
        1    0.000    0.000    0.742    0.742 <string>:1(<module>)
        1    0.000    0.000    0.742    0.742 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [20]:
%lprun -f np_black_scholes np_black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY, call, put)

Timer unit: 1e-06 s

Total time: 2.04363 s
File: <ipython-input-13-1782969c5f48>
Function: np_black_scholes at line 36

Line #      Hits         Time  Per Hit   % Time  Line Contents
    36                                           def np_black_scholes ( nopt, price, strike, t, rate, vol, call, put ):
    37         1            3      3.0      0.0      mr = -rate
    38         1            2      2.0      0.0      sig_sig_two = vol * vol * 2
    39                                               
    40    100001        78158      0.8      3.8      for i in range(nopt):
    41    100000        88134      0.9      4.3          P = price[i]
    42    100000        87195      0.9      4.3          S = strike[i]
    43    100000        87431      0.9      4.3          T = t[i]
    44                                                   
    45    100000       189266      1.9      9.3          a = np.log(P / S)
    46    100000        87023      0.9      4.3          b = T * mr
    47         

## _Why does this example take longer, even with performance libraries?_

When using NumPy, we need to unloop the function and utilize NumPy Arrays to achieve vectorization.  
Notice the Below function removes the loop, but now passes in the strike, price, and t as an _entire_ array

In [27]:
def vec_black_scholes (nopt, price, strike, t, rate, vol ):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = np.log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = np_invsqrt(z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * spec.erf(w1)
    d2 = 0.5 + 0.5 * spec.erf(w2)

    Se = np.exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se
    
    return (call, put)

## Run timeit, cProfile, and line_profiler to see what is happening

In [28]:
%timeit vec_black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

7.82 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
cProfile.run('vec_black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)')

         5 function calls in 0.009 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 <ipython-input-13-1782969c5f48>:33(<lambda>)
        1    0.008    0.008    0.009    0.009 <ipython-input-27-87e3027d4e67>:1(vec_black_scholes)
        1    0.000    0.000    0.009    0.009 <string>:1(<module>)
        1    0.000    0.000    0.009    0.009 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [31]:
%lprun -f vec_black_scholes vec_black_scholes(nopt, price, strike, t, RISK_FREE, VOLATILITY)

Timer unit: 1e-06 s

Total time: 0.008656 s
File: <ipython-input-27-87e3027d4e67>
Function: vec_black_scholes at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def vec_black_scholes (nopt, price, strike, t, rate, vol ):
     2         1            2      2.0      0.0      mr = -rate
     3         1            2      2.0      0.0      sig_sig_two = vol * vol * 2
     4                                           
     5         1            1      1.0      0.0      P = price
     6         1            0      0.0      0.0      S = strike
     7         1            1      1.0      0.0      T = t
     8                                           
     9         1         1486   1486.0     17.2      a = np.log(P / S)
    10         1          114    114.0      1.3      b = T * mr
    11                                           
    12         1           97     97.0      1.1      z = T * sig_sig_two
    13         1  